In [ ]:
!pip install transformers torch accelerate faiss-cpu pymupdf langchain sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 98.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
import fitz

def extrair_texto_pdf(arquivo_pdf):
    with fitz.open(arquivo_pdf) as doc:
        texto = ""
        for pagina in doc:
            texto += pagina.get_text("text") + "\n"
    return texto


pdf_path = "/content/ESP32-WROVER-E (8M).pdf"
texto_do_pdf = extrair_texto_pdf(pdf_path)

print(texto_do_pdf[:1000])  #conferindo os primeiros caracteres do arquivo enviado


ESP32-WROVER-E &
ESP32-WROVER-IE
Datasheet
Version 1.1
Espressif Systems
Copyright © 2020
www.espressif.com

About This Document
This document provides the specifications for the ESP32-WROVER-E and ESP32-WROVER-IE modules.
Document Updates
Please always refer to the latest version on https://www.espressif.com/en/support/download/documents.
Revision History
For revision history of this document, please refer to the last page.
Documentation Change Notification
Espressif provides email notifications to keep customers updated on changes to technical documentation. Please
subscribe at www.espressif.com/en/subscribe. Note that you need to update your subscription to receive notifi-
cations of new products you are not currently subscribed to.
Certification
Download certificates for Espressif products from www.espressif.com/en/certificates.
Disclaimer and Copyright Notice
Information in this document, including URL references, is subject to change without notice. THIS DOCUMENT IS
PROVIDED AS I

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


#📄 Artigos científicos / documentos técnicos → chunk_size = 800–1000, chunk_overlap = 100–200
#📜 Conversas / perguntas curtas → chunk_size = 300–500, chunk_overlap = 50–100
#📚 Textos muito grandes → chunk_size = 1500+ para evitar muitos chunks.

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = splitter.split_text(texto_do_pdf)

print(f"Total de chunks criados: {len(chunks)}")


Total de chunks criados: 61


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# importando modelo de embeddings da huggin face
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


db = FAISS.from_texts(chunks, embeddings_model) # Criação de  banco de dados vetorial FAISS

print("Base vetorial criada com sucesso!")


<ipython-input-5-f6efdcd60137>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Base vetorial criada com sucesso!


In [ ]:
#para acessar modelos LLM's da hugging face deve-se fazer o login na plataforma
from huggingface_hub import login

login(token="token") #token configurado como função Read


In [ ]:
from huggingface_hub import whoami

print(whoami())


{'type': 'user', 'id': '626d622118565bd9fad22fa2', 'name': 'Iury', 'fullname': 'Chagas', 'email': 'iurychagas25@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/782d3069f585108f2df56811a088c8cc.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'genai', 'role': 'read', 'createdAt': '2025-05-01T11:13:40.585Z'}}}


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

modelo = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Carregar o tokenizer para o modelo especifico
tokenizer = AutoTokenizer.from_pretrained(modelo)


model = AutoModelForCausalLM.from_pretrained(
    modelo,
    torch_dtype=torch.float16, 
    device_map="auto"  # Se houver GPU, ele usa automaticamente
)

#  pipeline para geração de respostas
pipeline_geracao = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


#### Utilização básica de RAG para o arquivo enviado

In [ ]:
from langchain.chains import RetrievalQA, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.chains import StuffDocumentsChain  # Import from langchain.chains instead
from langchain.prompts import PromptTemplate
from langchain.schema import Document

# Criar interface do Mistral no LangChain
llm_local = HuggingFacePipeline(pipeline=pipeline_geracao)

# prompt para combinação de contexto antes de responder a perguntas
prompt_template = """
Use os seguintes trechos de documentos para responder à pergunta do usuário. Responda como um jovem especialista na área do documento.
Se você não souber a resposta, apenas diga que não sabe, não tente inventar uma resposta. Se o contexto estiver em inglês, responda conforme o idioma do usuario

Contexto:
{context}

Pergunta:
{question}

Resposta:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Criação da cadeia para combinar documentos antes de enviar ao LLM
combine_documents_chain = StuffDocumentsChain(
    llm_chain=LLMChain(llm=llm_local, prompt=prompt),
    document_variable_name="context"
)

#pipeline de Perguntas e Respostas
qa_chain = RetrievalQA(
        retriever=db.as_retriever(),
        combine_documents_chain=combine_documents_chain
    )

# Perguntas mapeadas para teste do modelo
pergunta = "Como foram obtidas as prevalências de saúde para o Brasil e suas macrorregiões e quantas pessoas foram entrevistadas na Pesquisa Dimensões Sociais das Desigualdades (PDSD) de 2008?"
ask = "Qual metodologia foi seguida para determinar os RR para o cálculo da FPA e como as razões de chance (OR) foram transformadas em RR?"
ask2= "Qual foi a carga de diabetes atribuível ao sobrepeso, obesidade e excesso de peso no Brasil em 2008, e como essa carga se compara a outros países da América Latina?"
pergunta = "Qual a descrição do pin IO21"
resposta = qa_chain.run(pergunta)

print("\n🧠 Resposta do modelo:\n")
print(resposta)



🧠 Resposta do modelo:


Use os seguintes trechos de documentos para responder à pergunta do usuário. Responda como um jovem especialista em saúde.
Se você não souber a resposta, apenas diga que não sabe, não tente inventar uma resposta. Se o contexto estiver em inglês, responda conforme o idioma do usuario

Contexto:
3. Pin Deﬁnitions
3.
Pin Definitions
3.1
Pin Layout
1
2
3
4
5
6
7
8
9
10
11
12
13
14
GND
3V3
EN
SENSOR_VP
SENSOR_VN
IO34
IO35
IO32
IO33
IO25
IO26
IO27
IO14
IO12
15
16
17
18
19
GND
IO13
NC
NC
NC
38
37
36
35
34
33
32
31
30
29
28
27
26
25
GND
IO23
IO22
TXD0
RXD0
IO21
NC
IO19
IO18
IO5
IO17
IO16
IO4
IO0
Pin 39
GND
Keepout Zone
GND
GND
GND
GND
GND
GND
GND
GND
24
23
22
21
20
IO2
IO15
NC
NC
NC
Figure 3: Pin Layout (Top View)
3.2
Pin Description
The module has 38 pins. See pin definitions in Table 3.
Espressif Systems
4
Submit Documentation Feedback
ESP32-WROVER-E & ESP32-WROVER-IE Datasheet V1.1

3. Pin Deﬁnitions
Name
No.
Type
Function
IO21
33
I/O
GPIO21, VSPIHD, EMAC_TX_EN
RXD0